In [1]:
## IMPORTS
import pandas as pd
import numpy as np
from datetime import date, timedelta
import yahoo_fin.stock_info as yf

In [2]:
spList = yf.tickers_sp500()
today = date.today()
today = today.strftime('%Y-%m-%d')
dictionary = {}
for ticker in spList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-01",end_date=today, index_as_date = True, interval="1wk")
        df.to_pickle('stockData/S&P500/'+ticker+'.pkl')
        print(ticker)
        if ticker not in dictionary.values():
            industry = yf.get_company_info(ticker).loc['industry'][0]
            if industry not in dictionary.keys():
                dictionary[industry] = []
            dictionary[industry].append(ticker)
    except:
        print("stock not found")

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK-B
BRO
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDAY
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COST
CPB
CPRT
CRL
CRM
CSCO
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CTXS
CVS
CVX
CZR
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDS
FDX
FE
FFIV
FIS
FISV
FITB
FLT
FMC
FOX
FOXA
FRC
FRT
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [4]:
import pickle
try:
    industryDict = open('stockData/industries.pkl', 'wb')
    pickle.dump(dictionary, industryDict)
    industryDict.close()
  
except:
    print("Something went wrong")

['AAL', 'ALK', 'DAL', 'LUV', 'UAL']
